<a href="https://colab.research.google.com/github/alektebel/03MAIR--Algoritmos-de-optimizacion--2024/blob/main/Diego_Rodr%C3%ADguez_AG1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad guiada 1

In [ ]:
url_gh_AG1 = https://github.com/alektebel/03MAIR--Algoritmos-de-optimizacion--2024/tree/main/AG1

### Decorador para medir el tiempo de ejecución

In [12]:
def medir_tiempo(func):
  def wrapper(*args, **kwargs):
    import time
    inicio = time.time()
    resultado = func(*args, **kwargs)
    tiempo = (time.time() - inicio) *1000000
    print(f"Tiempo de ejecución: {tiempo:.2f} microseconds")
    return resultado
  return wrapper

Algoritmo de euclides

In [13]:
@medir_tiempo
def euclides(a, b):
  if b < a:
    b, a = a, b
  while(b):
    a, b = b, a % b
  return a

In [16]:
test_time = [(105000, 400000), (20000230, 10000650), (1870000000, 1000654000)]


for test in test_time:
  print(f"Test: {test}")
  euclides(*test)


Test: (105000, 400000)
Tiempo de ejecución: 3.34 microseconds
Test: (20000230, 10000650)
Tiempo de ejecución: 1.67 microseconds
Test: (1870000000, 1000654000)
Tiempo de ejecución: 3.10 microseconds


Algoritmo de heron

In [23]:
@medir_tiempo
def heron(x0, p, tol):
  while(abs(x0**2-p) > tol):
    x0 = (x0 + p / x0) / 2
  return x0

In [24]:
tests = [2, 3, 4, 5, 6]
tol = 0.001

import numpy as np

for test in tests:
  print(f"Test: {test}")
  print('La raiz de '+str(test) + ' -> ' +str(heron(1, test, tol)))
  print('La raiz real es: ', np.sqrt(test))

Test: 2
Tiempo de ejecución: 16.93 microseconds
La raiz de 2 -> 1.4142156862745097
La raiz real es:  1.4142135623730951
Test: 3
Tiempo de ejecución: 9.54 microseconds
La raiz de 3 -> 1.7321428571428572
La raiz real es:  1.7320508075688772
Test: 4
Tiempo de ejecución: 7.15 microseconds
La raiz de 4 -> 2.0000000929222947
La raiz real es:  2.0
Test: 5
Tiempo de ejecución: 5.72 microseconds
La raiz de 5 -> 2.2360688956433634
La raiz real es:  2.23606797749979
Test: 6
Tiempo de ejecución: 4.29 microseconds
La raiz de 6 -> 2.4494943716069653
La raiz real es:  2.449489742783178


## Problema de autobuses

Desarrollo e implementación de algoritmos. Ejemplo
Una pequeña ciudad estudia introducir un sistema de transporte urbano de
autobuses. Nos encargan estudiar el número mínimo de autobuses para
cubrir la demanda. Se ha realizado un estudio para estimar el número
mínimo de autobuses por franja horaria. Lógicamente este número varia
dependiendo de la hora del día. Se observa que es posible dividir la franja
horaria de 24h en tramos de 4 horas en los queda determinado el número
de autobuses que se necesitan. Debido a la normativa cada autobús debe
circular 8 horas como máximo y seguidas en cada jornada de 24h

In [25]:
req_buses = [8, 10, 7, 12, 6, 9]
tramos = len(req_buses)
x = [0]*tramos

Queremos minimizar el número de autobuses que necesitamos, lo que podemos expresar como el máximo entre los \(x_t\) donde estos son los autobuses utilizados. Además contamos con las condiciones antes expresadas, por lo que tenemos un problema de programación lineal:
$$
  min_x max_t x_t
$$
Las condiciones son
$$
x_t+x_{t+1} \leq r_t
$$
Donde $r_t$ son los autobuses requeridos en la franja de tiempo t.



In [3]:
#Una forma de resolver el problema a fuerza bruta sería:

#Probamos con un número alto de autobuses y vamos bajando el número, probando todas las combinaciones
#posibles de autobuses en los rangos.
n_autobuses = [i for i in range(12, 18)]

In [4]:
n_autobuses

[12, 13, 14, 15, 16, 17]

In [14]:
from itertools import combinations


def solucion_valida(solution, req_buses, num_buses):
    """
    Comprueba si una solución es válida
    """
    for i in range(len(req_buses)):
        if sum(solution[i:i+2]) < req_buses[i]:#Los buses deben estar de tal forma que se ajusten a la correspondiente demanda
            return False
    if sum(solution) > num_buses:#No puede utilizar más buses de los que tiene
        return False
    return True

def encontrar_solucion_brute_force(demanda, lb_buses = 15, up_buses = 30):
    """
    Encuentra alguna solución posible utilizando fuerza bruta
    """
    # Generar todos los posibles turnos válidos de 8 horas (2 franjas consecutivas)
    x = [0]*len(demanda)
    min_buses = float('inf')
    mejor_solucion = None

    for num_buses in range(lb_buses, up_buses+1):
      print('Buscando solución para ', num_buses, ' buses')
      for sol_c in range(num_buses**(len(demanda))):

        sol = []
        for _ in range(len(demanda)):
          sol.append(sol_c % num_buses)
          sol_c //= num_buses

        if solucion_valida(sol, demanda, num_buses):
          print('Se ha encontrado una solución valida con :', num_buses)
          if num_buses < min_buses:
            min_buses = num_buses
            mejor_solucion = sol
            print('La solucion es: ', sol)
      if min_buses == float('inf'):
        print('Ninguna solución para ', num_buses, ' buses.')
    return min_buses, mejor_solucion


# Ejemplo de uso
if __name__ == "__main__":
    # Ejemplo de demanda por franja horaria (00-04, 04-08, 08-12, 12-16, 16-20, 20-24)
    demand = [8, 10, 7, 12, 6, 9]

    min_buses, solucion = encontrar_solucion_brute_force(demand, 30, 35)


Buscando solución para  30  buses


KeyboardInterrupt: 

Si el numero de tramos es n, vemos que la complejidad de el algoritmo de evaluar todas las posibilidades crece de forma exponencial, concretamente, del orden de O(maxbuses^n) donde maxbuses sería el maximo numero de buses que estaríamos probando en el tramo. Obviamente hay algoritmos mejores que este, como verlo como un problema de programación lineal, y utilizar el algoritmo simplex.